<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/clause_label_tagging_vRG20230302.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 9.6 MB/s eta 0:00:00


In [5]:
import warnings
warnings.simplefilter("ignore")

import pandas as pd 
from unidecode import unidecode
import numpy as np
import json

In [6]:
def excel_keyword_parser( keyword_excel):
    '''
    Make dict for all keywords with respected clause label
    input : keyword excel
    output : return dict
    '''
    df = pd.read_excel(keyword_excel,sheet_name = 'Clauses')
    df_fr = df[['Clause Type in English','Keywords in Native Language']].copy()
    df_fr.iloc[:,1] = df_fr.iloc[:,1].str.replace(';',',').replace(['"','-'],' ',regex= True).str.split(',')
    clause_kw_dict = dict(list(zip(df_fr.iloc[:,0],df_fr.iloc[:,1])))
    return clause_kw_dict

In [7]:
#for matching 1 word
def match_word(word,clause):
    '''
    Match word with clause
    input : word from list of keyword and clause
    output : flag if keyword match
    '''
    count = 0
    if '&' in word:
        words = word.split('&')
        for j in words: 
            if clause.__contains__(j):
                count = count + 1
        flag = word if count == len(words) else None
    else:
        flag = word if clause.__contains__(word) else None
    return flag


#Matching a keywords list from clause 
def match_keyword_to_clause(keyword_list,clause):
    '''
    Match list of keywords with clause
    input : list of keyword and clause
    output : return sum of keyword present
    '''
    keyword = [unidecode(j.strip().lower()) for j in keyword_list]
    clause = unidecode(clause.lower().replace('-',' '))
    clause = unidecode(clause.lower().replace('\n',' '))
    cleaned_clause = ' '.join(clause.splitlines())
    key_match = []
    for i in keyword:
        key_match.append(match_word(i,cleaned_clause))
    match_key = list(filter(None.__ne__, key_match))
    return match_key

def para_clause_tagging(clause, clause_kw_dict):
    '''
    Match clause with keyword
    input : clause and dict of keyword
    output : return clause label with max keyword
    '''
    d = dict((k, match_keyword_to_clause(v,clause)) for k, v in clause_kw_dict.items())
    max_length = 0
    max_key_value = None

    for key, value in d.items():
        if isinstance(value, list) and len(value) > max_length:
            max_length = len(value)
            max_key_value = (key, value)

    return max_key_value     

## saving result as excel 

In [87]:
def get_clause_type_excel(excel_file,clause_kw_dict):
    df1 = pd.read_excel(excel_file)
    df1['section_id'].fillna('s0',inplace=True)

    #gives paragraph level clause type
    df1['para_level_clause_type'] = df1.iloc[:,1].apply(lambda x: para_clause_tagging(str(x),clause_kw_dict)[0] if para_clause_tagging(str(x),clause_kw_dict) is not None else None)
    df1['para_level_clause_type_keyword'] = df1.iloc[:,1].apply(lambda x: para_clause_tagging(str(x),clause_kw_dict)[1] if para_clause_tagging(str(x),clause_kw_dict) is not None else None)

    #combine text with identical section id
    sec1 = df1.groupby('section_id')['paragraph'].apply(lambda x: (' '.join(x))).reset_index()
 
    sec1['section_level_clause_type'] = sec1['paragraph'].apply(lambda x: para_clause_tagging(x,clause_kw_dict)[0] if para_clause_tagging(x,clause_kw_dict) is not None else None)
    sec1['section_level_clause_type_keyword'] = sec1['paragraph'].apply(lambda x: para_clause_tagging(x,clause_kw_dict)[1] if para_clause_tagging(x,clause_kw_dict) is not None else None)
    df_sec = pd.merge(df1,sec1.drop('paragraph', axis =1),on =['section_id'],how ='outer') 
    
    df_ab = df_sec.dropna(subset=['section_level_clause_type'], how='all')
    section = df_ab['section_id'].unique().tolist()
    result = pd.DataFrame(columns = ['section_id','clause_type','para_id','clause'])
    for i in section:
        test = df_ab[df_ab['section_id']==i]
        if test['para_level_clause_type'].nunique() ==1:
            if test['para_level_clause_type'].dropna().unique() == test['section_level_clause_type'].unique():
                id = test['para_id'].tolist()
                section_id = i
                clause = " ".join(test['paragraph'])
                clause_type = test['section_level_clause_type'].unique()
                result.loc[len(result.index)] = [section_id,clause_type,id,clause]
        else:
            test.dropna(subset=['para_level_clause_type'],how='all',inplace = True)
            test['kw_str'] = test.para_level_clause_type_keyword.apply(lambda x: ''.join(x))
            test['cond_match'] = np.where(test.para_level_clause_type != test.section_level_clause_type, np.where(test.kw_str.str.find('&'or ' ') ==-1,None, "keep"), "keep")
            test.dropna(subset=['cond_match'],how='all',inplace = True)
            g = test.groupby(['section_id','para_level_clause_type']).agg({'para_id':lambda x: x.tolist(),'paragraph': lambda x:' '.join(x)})
            g.reset_index(inplace= True)
            result = pd.concat([result,g.rename(columns={'paragraph':'clause','para_level_clause_type':'clause_type'})])
    result.to_excel("clause_with_label1.xlsx")

In [10]:
clause_kw_dict = excel_keyword_parser("Clause Management_Deep Dive France_Mapping Document.xlsx")
get_clause_type_excel('s04_cctp_montpellier v2_ocr1.xlsx',clause_kw_dict)

## saving result as json

In [29]:
def get_clause_type(excel_file,clause_kw_dict):
    df1 = pd.read_excel(excel_file)
    df1['section_id'].fillna('s0',inplace=True)

    #gives paragraph level clause type
    df1['para_level_clause_type'] = df1.iloc[:,1].apply(lambda x: para_clause_tagging(str(x),clause_kw_dict)[0] if para_clause_tagging(str(x),clause_kw_dict) is not None else None)
    df1['para_level_clause_type_keyword'] = df1.iloc[:,1].apply(lambda x: para_clause_tagging(str(x),clause_kw_dict)[1] if para_clause_tagging(str(x),clause_kw_dict) is not None else None)

    #combine text with identical section id
    sec1 = df1.groupby('section_id')['paragraph'].apply(lambda x: (' '.join(x))).reset_index()
 
    sec1['section_level_clause_type'] = sec1['paragraph'].apply(lambda x: para_clause_tagging(x,clause_kw_dict)[0] if para_clause_tagging(x,clause_kw_dict) is not None else None)
    sec1['section_level_clause_type_keyword'] = sec1['paragraph'].apply(lambda x: para_clause_tagging(x,clause_kw_dict)[1] if para_clause_tagging(x,clause_kw_dict) is not None else None)
    df_sec = pd.merge(df1,sec1.drop('paragraph', axis =1),on =['section_id'],how ='outer') 
    df_sec.to_excel("extract_clause_type_" + excel_file)

    df_ab = df_sec.dropna(subset=['section_level_clause_type'], how='all')
    section = df_ab['section_id'].unique().tolist()
    result = pd.DataFrame(columns = ['value','clause_type'])
    for i in section:
        test = df_ab[df_ab['section_id']==i]
        if test['para_level_clause_type'].nunique() ==1:
            if test['para_level_clause_type'].dropna().unique() == test['section_level_clause_type'].unique():
                clause = " ".join(test['paragraph'])
                clause_type = test['section_level_clause_type'].unique()[0]
                result.loc[len(result.index)] = [clause,clause_type]
        else:
            test.dropna(subset=['para_level_clause_type'],how='all',inplace = True)
            test['kw_str'] = test.para_level_clause_type_keyword.apply(lambda x: ''.join(x))
            test['cond_match'] = np.where(test.para_level_clause_type != test.section_level_clause_type, np.where(test.kw_str.str.find('&'or ' ') ==-1,None, "keep"), "keep")
            test.dropna(subset=['cond_match'],how='all',inplace = True)
            g = test.groupby(['para_level_clause_type']).agg({'paragraph': lambda x:' '.join(x)})
            g.reset_index(inplace= True)
            result = pd.concat([result,g.rename(columns={'paragraph':'value','para_level_clause_type':'clause_type'})],ignore_index = True)
    result.reset_index(inplace = True)
    result['confidence'] = 0
    result['file_name'] = excel_file
    out = result.to_dict(orient = 'records')
    with open('clauses.json','w') as f:
      json.dump({'clause': out},f, ensure_ascii=False)

In [30]:
clause_kw_dict = excel_keyword_parser("Clause Management_Deep Dive France_Mapping Document.xlsx")
get_clause_type('s04_cctp_montpellier v2_ocr1.xlsx',clause_kw_dict)
